In [1]:
from qiskit import *

from GraphMitigation import pairwise_mitigation_circuits, PairwiseMitigationFitter

In [2]:
N = 10

qc = QuantumCircuit(N)

strings = []
for circuit in pairwise_mitigation_circuits(qc):
    strings.append(execute(circuit,Aer.get_backend('qasm_simulator'),shots=1,memory=True).result().get_memory()[0])
     
for j in range(N-1):
    for k in range(j+1,N):
        substrings = []
        for string in strings:
            substring = string[j]+string[k]
            if substring not in substrings:
                substrings.append(substring)
        for substring in ['00','01','10','11']:
            assert(substring in substrings)

In [3]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p):

    error_meas = pauli_error([('X',p), ('I', 1 - p)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
        
    return noise_model

In [4]:
circuits = pairwise_mitigation_circuits(qc)
results = execute(circuits,Aer.get_backend('qasm_simulator'),noise_model=get_noise(0.1)).result()
fit = PairwiseMitigationFitter(results, circuits)
fit.mitigate_counts({'00':100,'01':10,'10':10},(0,1))

{'00': 120.0, '01': 0.0, '10': 0.0, '11': 0.0}